In [ ]:
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information numpy, numba, numexpr
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

print('\nMBP 14" [2024, macOS 15.2, M4Pro(8P+4E/G20c/N16c/48G)]')

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2025-05-29 22:54:26 (KST = GMT+0900) 
0 Python     3.11.11 64bit [Clang 18.1.8 ]
1 IPython    8.31.0
2 OS         macOS 15.2 arm64 arm 64bit
3 numpy      2.1.3
4 numba      0.61.0
5 numexpr    2.10.2

MBP 14" [2024, macOS 15.2, M4Pro(8P+4E/G20c/N16c/48G)]


In [41]:
import numba as nb
import numpy as np
import numexpr as ne

def iau_hg_model_ne(alpha__deg, gpar=0.15):
    """The IAU HG phase function model in intensity (1 at alpha=0).

    Parameters
    ----------
    alpha__deg : array_like
        The phase angle (Sun-Target-Observer angle) in degrees.

    gpar : float, optional
        The slope parameter ($G$) in the IAU H, G modeling. See Notes.
        By default ``0.15``.

    Notes
    -----
    See Bowell et al. 1989.
    """
    alpha__deg = np.abs(alpha__deg)
    alpha__rad = np.deg2rad(alpha__deg)
    return ne.evaluate(
        "(1 - gpar) * ("
        "exp(-90.56 * tan(alpha__rad * 0.5)**2) * (1 - 0.986 * (sin(alpha__rad) / (0.119 + 1.341 * sin(alpha__rad) - 0.754 * sin(alpha__rad)**2)))"
        "+ (1 - exp(-90.56 * tan(alpha__rad * 0.5)**2)) * exp(-3.332 * tan(alpha__rad * 0.5)**0.631)"
        ") + gpar * ("
        "exp(-90.56 * tan(alpha__rad * 0.5)**2) * (1 - 0.238 * (sin(alpha__rad) / (0.119 + 1.341 * sin(alpha__rad) - 0.754 * sin(alpha__rad)**2)))"
        "+ (1 - exp(-90.56 * tan(alpha__rad * 0.5)**2)) * exp(-1.862 * tan(alpha__rad * 0.5)**1.218)"
        ")",
        local_dict={"alpha__rad": alpha__rad, "gpar": gpar}
    )

_D2R = np.pi / 180.0

@nb.njit(fastmath=True, cache=True)
def iau_hg_model_nb(alpha, gpar=0.15):
    """
    Compute the two HG phase-function components for an array of phase angles.
    """
    n = alpha.shape[0]
    phi1 = np.empty(n, dtype=np.float64)
    phi2 = np.empty(n, dtype=np.float64)
    for i in range(n):
        # convert degrees to radians
        ar = np.abs(alpha[i]) * _D2R

        # intermediate trig and weighting terms
        sa = np.sin(ar)
        fa = sa / (0.119 + 1.341 * sa - 0.754 * sa * sa)
        tah = np.tan(ar * 0.5)
        w = np.exp(-90.56 * tah * tah)

        # smooth (s) and linear (l) components
        phi1_s = 1.0 - 0.986 * fa
        phi2_s = 1.0 - 0.238 * fa
        phi1_l = np.exp(-3.332 * np.pow(tah, 0.631))
        phi2_l = np.exp(-1.862 * np.pow(tah, 1.218))

        # mix them
        phi1[i] = w * phi1_s + (1.0 - w) * phi1_l
        phi2[i] = w * phi2_s + (1.0 - w) * phi2_l

    return (1 - gpar) * phi1 + gpar * phi2

In [42]:
def _hgphi12(alpha__deg):
    """Compute the HG phase function components.

    Parameters
    ----------
    alpha__deg : array_like
        The phase angle (Sun-Target-Observer angle) in degrees.

    Returns
    -------
    phi1, phi2 : tuple
        The two components of the HG phase function. See `iau_hg_model`
        for more details.
    """
    alpha__rad = np.deg2rad(alpha__deg)
    sin_a = np.sin(alpha__rad)
    f_a = sin_a / (0.119 + 1.341 * sin_a - 0.754 * sin_a * sin_a)
    tan_a_half = np.tan(alpha__rad * 0.5)
    w = np.exp(-90.56 * tan_a_half * tan_a_half)
    phi1_s = 1 - 0.986 * f_a
    phi2_s = 1 - 0.238 * f_a
    phi1_l = np.exp(-3.332 * tan_a_half**0.631)
    phi2_l = np.exp(-1.862 * tan_a_half**1.218)
    return (w * phi1_s + (1 - w) * phi1_l, w * phi2_s + (1 - w) * phi2_l)


def iau_hg_model_np(alpha__deg, gpar=0.15):
    """The IAU HG phase function model in intensity (1 at alpha=0).

    Parameters
    ----------
    alpha__deg : array_like
        The phase angle (Sun-Target-Observer angle) in degrees.

    gpar : float, optional
        The slope parameter ($G$) in the IAU H, G modeling. See Notes.
        By default ``0.15``.

    Notes
    -----
    Semi-empirical model of the phase function of the Moon, asteroids, and
    other (airless) solar system objects. The phase function defined at
    $(0^\circ \le \alpha \le 120^\circ)$ for the phase angle $\alpha$. It
    is given by the following equation:

    .. math::
        \Phi_\mathrm{HG}(\alpha, G) = G \Phi_{HG1}(\alpha) + (1-G) \Phi_{HG2}(\alpha)

    where

    .. math::
        \Phi_{HG i}(\alpha) = W \left ( 1-\frac{C_i \sin \alpha}{0.119+1.341 \sin \alpha-0.754 \sin ^2 \alpha} \right )
        + (1 - W) \times \exp \left \{ -A_i \left [ \tan \frac{\alpha}{2} \right ]^{B_i} \right \}

    and

    .. math::
        W(\alpha) = \exp \left \{ -90.56 \tan^2 \frac{\alpha}{2} \right \}

    The parameters $A_i$, $B_i$, and $C_i$ are given by:

    .. math::
        A_1, A_2 &= 3.332, 1.862 \sep
        B_1, B_2 = 0.631, 1.218 \sep
        C_1, C_2 = 0.986, 0.238

    Reference: Bowell et al. 1989
    https://ui.adsabs.harvard.edu/abs/1989aste.conf..524B/abstract
    """
    hgphi1, hgphi2 = _hgphi12(np.array(np.abs(alpha__deg)))
    # Just to avoid negative alpha error
    return (1 - gpar) * hgphi1 + gpar * hgphi2

In [43]:
for length in [1, 10, 100, 1000]:
    alphas = np.linspace(0, 120, length)
    res1 = iau_hg_model_np(alphas)
    res2 = iau_hg_model_nb(alphas)
    res3 = iau_hg_model_ne(alphas)
    np.testing.assert_allclose(res1, res2, rtol=1e-5, atol=1.e-8)
    np.testing.assert_allclose(res2, res3, rtol=1e-5, atol=1.e-8)
    print(f"Test passed for length {length}.")
    %timeit -n 1000 -r 10 iau_hg_model_np(alphas)
    %timeit -n 1000 -r 10 iau_hg_model_ne(alphas)
    %timeit -n 1000 -r 10 iau_hg_model_nb(alphas)

Test passed for length 1.
9.35 μs ± 116 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
4.45 μs ± 153 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
386 ns ± 29 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
Test passed for length 10.
9.7 μs ± 239 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
4.81 μs ± 184 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
642 ns ± 23.3 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
Test passed for length 100.
11.9 μs ± 232 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
7.28 μs ± 211 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
3.14 μs ± 132 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
Test passed for length 1000.
34.6 μs ± 273 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
31.2 μs ± 357 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
28.2 μs ± 445 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
